In [1]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import sys
import http.client, urllib.request, urllib.parse, urllib.error, base64
from scipy import stats

from sklearn.model_selection import GridSearchCV

#Change root to the folder where you have the data stored, AR has the group project
root = r"G:/.shortcut-targets-by-id/1hgVHDpl_iZ7es6sKW0QRlC9X6SF58wk7/Machine Learning 4 Cities Final Project"

#Change this to match your folder structure
inp= os.join(root, "Data", "3_clean_data")
out= os.join(root, "Results")


In [2]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [3]:
# files in clean data google drive
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1jxSI_nq32-W9kPHKvQkzuNGJ5aIBD2j2' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: analysis_box.csv, ID: 1Vzjvta98IG9Hg4SfAGcd8JD8FaF3IXwy
Title: analysis_bbl.csv, ID: 1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G
Title: analysis_bbl_noacs.csv, ID: 1h-tDpBSslfzqzsJe1ny_ujDnr68RKry3
Title: borobox_convex_hull.geojson, ID: 1SzBNj3Ba-v9MEwuCkZT9e9iAwVYUqx8r


In [4]:
# load codebook        
codebk = drive.CreateFile({'id': '1Lf7kV9_R-caFZV6_l8hMxAVjFSVFhvYl'})
codebk.GetContentFile('codebook.xlsx')  

codebk= pd.read_excel('codebook.xlsx', sheet_name = "predictors")
codebk.head()

C:\Users\rivera30\Anaconda3\envs\geo_env\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Variable,Description,Source,Notes,Include_BBL,Include_Box
0,geofips,census tract geo identifier,Census ACS,NaN,0.0,0.0
1,geofips_recode,"census tract geo identifier, changes county to...",Census ACS,NaN,0.0,0.0
2,totpop,total population,Census ACS,NaN,1.0,1.0
3,popdens,total population per square mile,Census ACS,NaN,1.0,1.0
4,areasqmile,area,Census ACS,NaN,1.0,1.0


In [5]:
predictors_full = codebk.loc[codebk["Include_BBL"] == 1, "Variable"].apply(lambda x: x.strip())
predictors_full

2             totpop
3            popdens
4         areasqmile
5              males
6            females
           ...      
102        assesstot
103        exempttot
104        yearbuilt
105      landuse_cat
106    bldgclass_cat
Name: Variable, Length: 90, dtype: object

In [6]:
df_bbl = drive.CreateFile({'id': '1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G'})
df_bbl.GetContentFile('analysis_bbl.csv')  

df_bbl= pd.read_csv('analysis_bbl.csv').drop(["Unnamed: 0"], axis = 1)
df_bbl.head()

,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,...,bldgclass_A0,bldgclass_A6,bldgclass_A8,bldgclass_CM,bbl,boroct2010,vacate_ind,borough,landuse,bldgclass
0,2013.0,5555.0,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0,...,0.0,0.0,0.0,0.0,1012090027,1018100,0.0,MN,2.0,C2
1,2517.0,6895.0,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0,...,0.0,0.0,0.0,0.0,1012090028,1018100,0.0,MN,2.0,C5
2,10570.0,127289.0,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0,...,0.0,0.0,0.0,0.0,1012090037,1018100,0.0,MN,3.0,D4
3,3926.0,20358.0,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0,...,0.0,0.0,0.0,0.0,1012090049,1018100,0.0,MN,3.0,D4
4,5035.0,21258.0,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0,...,0.0,0.0,0.0,0.0,1012090051,1018100,0.0,MN,3.0,D4


In [7]:
assert df_bbl["bbl"].isna().sum() == 0
assert df_bbl["holc_D"].isna().sum() == 0

In [8]:
assert df_bbl.loc[:, "vacate_ind"].isna().sum() == 0

In [9]:
print(df_bbl.columns)
df_bbl.describe()

Index(['lotarea', 'bldgarea', 'numbldgs', 'numfloors', 'unitsres',
       'unitstotal', 'assessland', 'assesstot', 'exempttot', 'yearbuilt',
       ...
       'bldgclass_A0', 'bldgclass_A6', 'bldgclass_A8', 'bldgclass_CM', 'bbl',
       'boroct2010', 'vacate_ind', 'borough', 'landuse', 'bldgclass'],
      dtype='object', length=160)


,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,...,bldgclass_Z0,bldgclass_A2,bldgclass_A0,bldgclass_A6,bldgclass_A8,bldgclass_CM,bbl,boroct2010,vacate_ind,landuse
count,7.650260e+05,7.650260e+05,765026.000000,765026.000000,765026.000000,765026.000000,7.650260e+05,7.650260e+05,7.650260e+05,765026.000000,...,765026.000000,765026.000000,765026.000000,765026.000000,765026.000000,765026.000000,7.650260e+05,7.650260e+05,765026.000000,765026.000000
mean,3.812014e+03,5.051030e+03,1.324975,2.443933,4.689669,4.894059,5.407344e+04,2.821186e+05,5.387875e+04,1938.547234,...,0.000508,0.058912,0.011311,0.001029,0.000038,0.000001,3.548735e+09,3.541768e+06,0.003404,1.421477
std,2.318736e+04,3.520154e+04,2.726640,1.676083,42.058995,47.000007,5.127009e+05,2.549800e+06,1.355497e+06,68.722810,...,0.022544,0.235460,0.105749,0.032057,0.006157,0.001143,1.003515e+09,9.994487e+05,0.058243,0.842764
min,2.300000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000070e+09,1.000201e+06,0.000000,1.000000
25%,2.000000e+03,1.440000e+03,1.000000,2.000000,1.000000,1.000000,1.020000e+04,3.786000e+04,0.000000e+00,1920.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.036730e+09,3.030100e+06,0.000000,1.000000
50%,2.500000e+03,2.020000e+03,1.000000,2.000000,2.000000,2.000000,1.368000e+04,5.172000e+04,0.000000e+00,1931.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.012680e+09,4.010200e+06,0.000000,1.000000
75%,4.000000e+03,2.923000e+03,2.000000,2.750000,2.000000,2.000000,1.986000e+04,8.304000e+04,1.440000e+03,1960.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.107290e+09,4.092200e+06,0.000000,2.000000
max,1.225343e+07,1.354011e+07,1807.000000,98.000000,19201.000000,19203.000000,1.345064e+08,6.899244e+08,4.435560e+08,2021.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.080500e+09,5.032300e+06,1.000000,4.000000


In [10]:
df_bbl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765026 entries, 0 to 765025
Columns: 160 entries, lotarea to bldgclass
dtypes: float64(154), int64(4), object(2)
memory usage: 933.9+ MB


In [11]:
df_bbl= pd.get_dummies(df_bbl, columns=["borough"])
borocols= ["borough_" + x for x in ["BK", "BX", "MN", "QN", "SI"]]

In [12]:
len(df_bbl) 


765026

In [13]:

#print(df.nunique())
print(df_bbl['vacate_ind'].value_counts())

0.0    762422
1.0      2604
Name: vacate_ind, dtype: int64


In [14]:
df_bbl.loc[:, predictors_full].head()

,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,pop65plus,pophispanic,...,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,landuse_cat,bldgclass_cat
0,8496.0,158948.7,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0,2.0,0
1,8496.0,158948.7,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0,2.0,1
2,8496.0,158948.7,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0,3.0,2
3,8496.0,158948.7,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0,3.0,2
4,8496.0,158948.7,0.053451,4072.0,4424.0,1037.0,2010.0,3202.0,2247.0,1400.0,...,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0,3.0,2


In [15]:
outcomes= ["vacate_ind"]
df_bbl[outcomes].head()

# very small percentage of the data has had a fire
df_bbl[outcomes[0]].mean()

0.003403805883721599

In [17]:
predictors_fimp=["hpd_b_c_vio", "hpd_fire_vio", "landuse_cat", "popdens", "unitsres", "pcthoilker", "bldgarea", 
                 "pophispanic", "medianhhincome", "hpd_b_c_vio_per_unit", "average_energy_usage_mi" ] 


In [25]:
df_bbl_zeros = df_bbl.loc[df_bbl['vacate_ind'] == 0]
df_bbl_one = df_bbl.loc[df_bbl['vacate_ind'] == 1]

In [26]:
df_bbl_zeros.info()
df_bbl_one.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 762422 entries, 0 to 765025
Columns: 164 entries, lotarea to borough_SI
dtypes: float64(154), int64(4), object(1), uint8(5)
memory usage: 934.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2604 entries, 20 to 765015
Columns: 164 entries, lotarea to borough_SI
dtypes: float64(154), int64(4), object(1), uint8(5)
memory usage: 3.2+ MB


In [27]:
df_bbl_zeros_sample = df_bbl_zeros.sample(frac =.25, random_state=1234)

In [28]:
df_bbl_zeros_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190606 entries, 300801 to 67669
Columns: 164 entries, lotarea to borough_SI
dtypes: float64(154), int64(4), object(1), uint8(5)
memory usage: 233.6+ MB


In [29]:
df_bbl_ds = df_bbl_zeros_sample.append(df_bbl_one, ignore_index=True)

C:\Users\rivera30\AppData\Local\Temp\ipykernel_17756\299408441.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_bbl_ds = df_bbl_zeros_sample.append(df_bbl_one, ignore_index=True)


In [30]:
df_bbl_ds.reset_index

<bound method DataFrame.reset_index of          lotarea   bldgarea  numbldgs  numfloors  unitsres  unitstotal  \
0         2248.0     2280.0       1.0        2.0       3.0         3.0   
1         1400.0     6375.0       1.0        5.0       9.0        10.0   
2         2342.0     1536.0       1.0        2.5       1.0         1.0   
3         2433.0     2232.0       2.0        2.0       2.0         2.0   
4         2600.0     1410.0       1.0        2.0       1.0         1.0   
...          ...        ...       ...        ...       ...         ...   
193205  289064.0   366642.0       3.0        6.0     380.0       380.0   
193206  291220.0   352800.0       3.0        6.0     385.0       385.0   
193207  145000.0    67000.0      12.0        2.0     108.0       108.0   
193208   58934.0    44928.0      12.0        2.0      45.0        45.0   
193209  589394.0  1085148.0      37.0       19.0    1091.0      1091.0   

        assessland   assesstot   exempttot  yearbuilt  ...         bbl  

In [31]:
print(df_bbl_ds['vacate_ind'].value_counts())

0.0    190606
1.0      2604
Name: vacate_ind, dtype: int64


### Use Naive Bayes Classifier to predict whether a building has a vacate order or not

In [17]:
# Training a binary Naive Bayes Classifier with discrete input attributes.
# Assume that the binary output variable takes on values 0 or 1. 
#def trainNaiveBayes(X,y):
#    prior = 1.*y.sum()/y.count()
#    nbc = {'prior':prior}
#    X_1 = X[y==1]
#    X_0 = X[y==0]
    
#    for j in X.columns:
#        if X[j].dtype == "int":
#            nbc[j+'_1'] = X_1[j].value_counts(normalize=True)
#           nbc[j+'_0'] = X_0[j].value_counts(normalize=True)
#        else:    
#            nbc[j+'_mu1'] = X_1[j].mean()
#            nbc[j+'_sigma1'] = X_1[j].std()
#            nbc[j+'_mu0'] = X_0[j].mean()
#            nbc[j+'_sigma0'] = X_0[j].std()
#    return nbc

#mean normal 

#def testNaiveBayes(X,nbc):
#    y_pred = pd.Series(index=X.index,dtype='float64')
#    for i in X.index:
        # compute odds of y=1
#        y_pred[i] = nbc['prior']/(1-nbc['prior']) # prior odds
#        for j in X.columns:
#            if X[j].dtype == "int": 
#                thevalue = X.loc[i,j]
#                if thevalue not in nbc[j+'_1']:
#                    y_pred[i] = y_pred[i]*1E-3
#                if thevalue not in nbc[j+'_0']:
#                    y_pred[i] = y_pred[i]*1E3
#                if (thevalue in nbc[j+'_1']) & (thevalue in nbc[j+'_0']):
#                    y_pred[i] = y_pred[i]*(nbc[j+'_1'][thevalue]+1E-3)/(nbc[j+'_0'][thevalue]+1E-3)
#            else:
#               thevalue = X.loc[i,j]
#                pdf1 = stats.norm.pdf(thevalue,loc=nbc[j+'_mu1'],scale=nbc[j+'_sigma1'])
#                pdf0 = stats.norm.pdf(thevalue,loc=nbc[j+'_mu0'],scale=nbc[j+'_sigma0'])
#                y_pred[i] = y_pred[i]*pdf1/pdf0 if pdf0 > 0 else 1E10
        # convert odds to probability of y=1
#        y_pred[i] = y_pred[i]/(1.0+y_pred[i])
#    return y_pred

In [35]:
from sklearn.model_selection import train_test_split
#data1_X = data1.iloc[:,1:] # tip, distance, speed, and number of passengers
#data1_y = data1.iloc[:,0] # binary output: 1 if in Manhattan, 0 if outside

X = df_bbl.loc[:, predictors_fimp].fillna(0)
y = df_bbl.loc[:, "vacate_ind"].fillna(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)

In [36]:
def trainGaussianNaiveBayes(X,y):
    prior = 1.*y.sum()/y.count()
    nbc = {'prior':prior}
    X_1 = X[y==1]
    X_0 = X[y==0]
    for j in X.columns:
        nbc[j+'_mu1'] = X_1[j].mean()
        nbc[j+'_sigma1'] = X_1[j].std()
        nbc[j+'_mu0'] = X_0[j].mean()
        nbc[j+'_sigma0'] = X_0[j].std()
    return nbc

def testGaussianNaiveBayes(X,nbc):
    y_pred = pd.Series(index=X.index,dtype='float64')
    for i in X.index:
        # compute odds of y=1
        y_pred[i] = nbc['prior']/(1-nbc['prior']) # prior odds
        for j in X.columns:
            thevalue = X.loc[i,j]
            pdf1 = stats.norm.pdf(thevalue,loc=nbc[j+'_mu1'],scale=nbc[j+'_sigma1'])
            pdf0 = stats.norm.pdf(thevalue,loc=nbc[j+'_mu0'],scale=nbc[j+'_sigma0'])
            y_pred[i] = y_pred[i]*pdf1/pdf0 if pdf0 > 0 else 1E10
        # convert odds to probability of y=1
        y_pred[i] = y_pred[i]/(1.0+y_pred[i])
    return y_pred

In [46]:
naive_bayes_classifier = trainGaussianNaiveBayes(X_train,y_train)
for i,j in naive_bayes_classifier.items():
    print(i)
    print(j)
    print()
#y_pred_train = testGaussianNaiveBayes(X_train,naive_bayes_classifier)
y_pred_test = testGaussianNaiveBayes(X_test.head(20000),naive_bayes_classifier)

# measure accuracy for the binary prediction task
#print('In sample prediction accuracy:',1.0*sum((y_pred_train>0.5)==y_train)/len(y_train))
print('Out of sample prediction accuracy:',1.0*sum((y_pred_test>0.5)==y_test.head(20000))/len(y_test.head(20000)))

prior
0.003414266019948795

hpd_b_c_vio_mu1
181.5375191424196

hpd_b_c_vio_sigma1
303.32400833237455

hpd_b_c_vio_mu0
4.232757384445883

hpd_b_c_vio_sigma0
34.08121369728853

hpd_fire_vio_mu1
2.222052067381317

hpd_fire_vio_sigma1
4.378796024298838

hpd_fire_vio_mu0
0.054355467725293365

hpd_fire_vio_sigma0
0.5406894970463929

landuse_cat_mu1
2.8162327718223583

landuse_cat_sigma1
0.8587243186805446

landuse_cat_mu0
1.416007065283923

landuse_cat_sigma0
0.837834123768971

popdens_mu1
79310.10828381803

popdens_sigma1
40070.06797919319

popdens_mu0
38434.71788643216

popdens_sigma0
27636.48663045158

unitsres_mu1
73.80551301684532

unitsres_sigma1
309.14188367035575

unitsres_mu0
4.462678162326647

unitsres_sigma0
41.81174313897312

pcthoilker_mu1
25.323240091230325

pcthoilker_sigma1
18.802735256809648

pcthoilker_mu0
11.954517565996928

pcthoilker_sigma0
10.91493840138927

bldgarea_mu1
70206.2358346095

bldgarea_sigma1
272583.81216820393

bldgarea_mu0
4817.510947692415

bldgarea_sigma

C:\Users\rivera30\AppData\Local\Temp\ipykernel_17756\1594424590.py:22: RuntimeWarning: overflow encountered in double_scalars
  y_pred[i] = y_pred[i]*pdf1/pdf0 if pdf0 > 0 else 1E10
C:\Users\rivera30\AppData\Local\Temp\ipykernel_17756\1594424590.py:24: RuntimeWarning: invalid value encountered in double_scalars
  y_pred[i] = y_pred[i]/(1.0+y_pred[i])


Out of sample prediction accuracy: 0.9652


In [47]:
# measure accuracy for the binary prediction task
print('Out of sample prediction accuracy:',1.0*sum((y_pred_test>0.5)==y_test.head(20000))/len(y_test.head(20000)))

# measure accuracy of the predicted probabilities
print('Log-likelihood (test):', np.log(y_pred_test*y_test.head(20000) + (1-y_pred_test)*(1-y_test.head(20000))).replace([np.inf, -np.inf], np.nan).sum())

Out of sample prediction accuracy: 0.9652
Log-likelihood (test): -6940.9396602748075


C:\Users\rivera30\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [48]:
#Calculate precision and recall
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test.head(20000), (y_pred_test > 0.5))

(array([0.99870325, 0.06934813]),
 array([0.96632371, 0.66666667]),
 array([0.98224671, 0.12562814]),
 array([19925,    75], dtype=int64))

In [64]:
# count zeros in 1d array
y_pred_test = y_pred_test.replace(np.nan, 0).replace(-np.inf, 0)


#n_zeros = np.count_nonzero(np.isnan(y_pred_test))
# display the count of zeros
#print(n_zeros)

#np.count_nonzero(np.isnan(data))

In [65]:
# count zeros in 1d array
n_zeros = np.count_nonzero(np.isnan(y_pred_test))
# display the count of zeros
print(n_zeros)

#np.count_nonzero(np.isnan(data))

0


In [66]:
from sklearn.metrics import roc_auc_score

# predict_proba predicts the probability of each class rather than just the most likely class

print("ROC AUC:", roc_auc_score(np.array(y_test.head(20000)),y_pred_test))

ROC AUC: 0.9382772061898788
